## Analise de Dados - Empresa Zero Fome


### Contexto do negócio

Você acaba de ser contratado como Cientista de Dados da empresa
Fome Zero, e a sua principal tarefa nesse momento é ajudar o CEO Kleiton Guerra
a identificar pontos chaves da empresa, respondendo às perguntas que ele fizer
utilizando dados!
A empresa Fome Zero é uma marketplace de restaurantes. Ou seja, seu core
business é facilitar o encontro e negociações de clientes e restaurantes. Os
restaurantes fazem o cadastro dentro da plataforma da Fome Zero, que disponibiliza
informações como endereço, tipo de culinária servida, se possui reservas, se faz
entregas e também uma nota de avaliação dos serviços e produtos do restaurante,
dentre outras informações.

### Problema de negócio

O CEO Guerra também foi recém contratado e precisa entender melhor o negócio
para conseguir tomar as melhores decisões estratégicas e alavancar ainda mais a
Fome Zero, e para isso, ele precisa que seja feita uma análise nos dados da
empresa e que sejam gerados dashboards, a partir dessas análises, para responder
algumas perguntas.

#### Descrição das colunas

**Restaurant ID**:

**Restaurant Name**:

**Country Code**:

**City**:

**Address**:

**Locality**:

**Locality Verbose**:

**Longitude**:

**Latitude**:

**Cuisines**:

**Average Cost for two**:

**Currency**:

**Has Table booking**:

**Has Online delivery**:

**Is delivering now**:

**Switch to order menu**:

**Price range**:

**Aggregate rating**:

**Rating color**:

**Rating text**:

**Votes**:

### Importando bibliotecas e Funções

In [282]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import inflection

#### Funções

In [283]:
def create_price_tye(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"

def rename_columns(dataframe):
    df = dataframe.copy()
    title = lambda x: inflection.titleize(x) #converte a primiera de cada palavra em maiúscula
    snakecase = lambda x: inflection.underscore(x) #transformar de camelcase para snakecase
    spaces = lambda x: x.replace(" ", "") #retira o espaço entre palavras
    cols_old = list(df.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    return df


#### Loading data

In [284]:
df = pd.read_csv('../data/zomato.csv')

### Descrevendo os dados

In [285]:
print(f"O dataset tem {df.shape[0]} linhas e {df.shape[1]} colunas.")

O dataset tem 7527 linhas e 21 colunas.


In [286]:
df.head(2)

Restaurant ID             Restaurant Name  Country Code            City  \
0        6310675  Mama Lou's Italian Kitchen           162  Las Piñas City   
1        6310675  Mama Lou's Italian Kitchen           162  Las Piñas City   

                                             Address          Locality  \
0  Block 1, Lot 36, Tropical Avenue Corner Tropic...  BF International   
1  Block 1, Lot 36, Tropical Avenue Corner Tropic...  BF International   

                   Locality Verbose   Longitude   Latitude Cuisines  ...  \
0  BF International, Las Piñas City  121.009787  14.447615  Italian  ...   
1  BF International, Las Piñas City  121.009787  14.447615  Italian  ...   

           Currency Has Table booking  Has Online delivery  Is delivering now  \
0  Botswana Pula(P)                 1                    0                  0   
1  Botswana Pula(P)                 1                    0                  0   

   Switch to order menu  Price range  Aggregate rating  Rating color  \
0                     0            3               4.6        3F7E00   
1                     0            3               4.6        3F7E00   

  Rating text Votes  
0   Excellent   619  
1   Excellent   619  

[2 rows x 21 columns]

In [287]:
df.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')

#### Rename columns

In [288]:
df = rename_columns(df)
df.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes'],
      dtype='object')

#### Feature Engineering

In [289]:
df1 = df.copy()

In [290]:
# colocando nomes nos paises de acordo com o id, criando nova coluna com nomes dos paises

COUNTRIES = {
1: "India",
14: "Australia",
30: "Brazil",
37: "Canada",
94: "Indonesia",
148: "New Zeland",
162: "Philippines",
166: "Qatar",
184: "Singapure",
189: "South Africa",
191: "Sri Lanka",
208: "Turkey",
214: "United Arab Emirates",
215: "England",
216: "United States of America",
}

df1['country_name'] = df1['country_code'].map(COUNTRIES)
df1['country_name'].head(3)


0    Philippines
1    Philippines
2    Philippines
Name: country_name, dtype: object

In [291]:
# colocando nomes para os tipos de preço de acordo com a classificação
df1['price_range'] = df1['price_range'].apply(create_price_tye)
df1['price_range'].head(3)

0    expensive
1    expensive
2      gourmet
Name: price_range, dtype: object

In [292]:
# tranformando código das cores para o nome

COLORS = {
"3F7E00": "darkgreen",
"5BA829": "green",
"9ACD32": "lightgreen",
"CDD614": "orange",
"FFBA00": "red",
"CBCBC8": "darkred",
"FF7800": "darkred",
}

df1['rating_color'] = df1['rating_color'].map(COLORS)
df1['rating_color'].head(3)


0    darkgreen
1    darkgreen
2    darkgreen
Name: rating_color, dtype: object

In [293]:
# definindo apenas um tipo de cozinha por restaurante 

df1["cuisines"] = df1["cuisines"].apply(lambda x: str(x).split(",")[0]) # não entendi pq ele colocou como float e eu precisei comverter para string
df1['cuisines'].head(5)

0     Italian
1     Italian
2    European
3    Filipino
4    American
Name: cuisines, dtype: object

#### Data types

In [176]:
df1.dtypes

restaurant_id             int64
restaurant_name          object
country_code              int64
city                     object
address                  object
locality                 object
locality_verbose         object
longitude               float64
latitude                float64
cuisines                 object
average_cost_for_two      int64
currency                 object
has_table_booking         int64
has_online_delivery       int64
is_delivering_now         int64
switch_to_order_menu      int64
price_range              object
aggregate_rating        float64
rating_color             object
rating_text              object
votes                     int64
country_name             object
dtype: object

#### Check NA

In [294]:
df1.isna().sum()

restaurant_id           0
restaurant_name         0
country_code            0
city                    0
address                 0
locality                0
locality_verbose        0
longitude               0
latitude                0
cuisines                0
average_cost_for_two    0
currency                0
has_table_booking       0
has_online_delivery     0
is_delivering_now       0
switch_to_order_menu    0
price_range             0
aggregate_rating        0
rating_color            0
rating_text             0
votes                   0
country_name            0
dtype: int64

#### Drop NA

In [295]:
#excluindo na
df1 = df1.dropna()

#checando na novamente
df1.isna().sum()

restaurant_id           0
restaurant_name         0
country_code            0
city                    0
address                 0
locality                0
locality_verbose        0
longitude               0
latitude                0
cuisines                0
average_cost_for_two    0
currency                0
has_table_booking       0
has_online_delivery     0
is_delivering_now       0
switch_to_order_menu    0
price_range             0
aggregate_rating        0
rating_color            0
rating_text             0
votes                   0
country_name            0
dtype: int64

#### Check duplicates

In [296]:
n_duplicated = df1.duplicated().sum()

print(f"Temos {n_duplicated} linhas duplicadas no dataset.")

Temos 585 linhas duplicadas no dataset.


#### Drop duplicates

In [297]:
#excluindo linhas duplicadas
df1 = df1.drop_duplicates()

#checando número de linhas duplicadas novamente
n_duplicated = df1.duplicated().sum()

print(f"Temos {n_duplicated} linhas duplicadas no dataset.")

Temos 0 linhas duplicadas no dataset.


#### Estatítica descritiva

Como podemos ver acima, diversos atributos são do tipo int64 e float64, entretando nem todos esses atributos podem ser calculados usando média, médiana e outros métodos estatíticos. Isso occore pois muitos são valores booleanos (0 ou 1) e não indicam de fato um atributo número. Outros se referem ao "id" das lojas e dos países. Outros, descrevem a latitute e longitude da loja em questão. Por fim, ficamos com apenas 3 atributos númericos: Average Cost for two (custo médio para duas pessoas), Aggregate rating (quantas estrelas aquele restuarante recebeu) e Votes (a quantidade de votos recebida). Entretando, o custo médio para duas pessoas estão em 12 moedas diferentes, sendo necesário analisar cada moeda separadamente para termos valores reais na estatítica descritiva. Por isso, escolhi descrever inicialmente somente as últimas duas colunas citadas: Aggregate rating (quantas estrelas aquele restuarante recebeu) e Votes (a quantidade de votos recebida).

#### Variáveis numéricas

In [298]:
num_attributes = df[['Aggregate rating', 'Votes']]

KeyError: "None of [Index(['Aggregate rating', 'Votes'], dtype='object')] are in the [columns]"

In [299]:
## Numérical atributes

# Central Tendency - mean, median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

# Dispersion - std, min, max, range, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(min)).T
d3 = pd.DataFrame(num_attributes.apply(max)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

#concatenate
m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']

m


attributes  min      max    range        mean  median          std  \
0  Aggregate rating  0.0      4.9      4.9    4.118055     4.2     0.680146   
1             Votes  0.0  41333.0  41333.0  616.401488   335.0  1127.623019   

        skew    kurtosis  
0  -3.888787   20.607593  
1  10.668222  260.759775

#### Variáveis categóricas

In [300]:
cat_attributes = df1.select_dtypes(exclude = ['int', 'float64'])
cat_attributes.columns

Index(['restaurant_name', 'city', 'address', 'locality', 'locality_verbose',
       'cuisines', 'currency', 'price_range', 'rating_color', 'rating_text',
       'country_name'],
      dtype='object')

In [301]:
cat_attributes.apply(lambda x: x.nunique())

restaurant_name     5914
city                 125
address             6760
locality            2272
locality_verbose    2357
cuisines             166
currency              12
price_range            4
rating_color           6
rating_text           28
country_name          15
dtype: int64

### Analise dos dados - respondendo perguntas 

In [302]:
df2 = df1.copy()

#### Geral


1. Quantos restaurantes únicos estão registrados?

In [303]:
n_rest = df2['restaurant_id'].nunique()

print(f"O número de restaurantes únicos é de {n_rest}.")

O número de restaurantes únicos é de 6942.


2. Quantos países únicos estão registrados?

In [304]:
n_country = df2['country_name'].nunique()

print(f"O número de países únicos é de {n_country}.")

O número de países únicos é de 15.


3. Quantas cidades únicas estão registradas?

In [305]:
n_city = df2['city'].nunique()

print(f"O número de cidades únicos é de {n_city}.")

O número de cidades únicos é de 125.


4. Qual o total de avaliações feitas?

In [306]:
n_rating = df['aggregate_rating'][df['aggregate_rating'] != 0].count()

print(f"O número de avaliações totais é de {n_rating}.")

O número de avaliações totais é de 7398.


5. Qual o total de tipos de culinária registrados?

In [307]:
n_cuisines = df2['cuisines'].nunique()

print(f"O número de tipos de culinária registrados é de {n_cuisines}.")

O número de tipos de culinária registrados é de 166.


#### Países

1. Qual o nome do país que possui mais cidades registradas?

In [308]:
max_country_city = df2[['country_name', 'city']].groupby('country_name')['city'].nunique().idxmax()

print(f"O país que possui mais cidades registradas é {max_country_city}.")

O país que possui mais cidades registradas é India.


2. Qual o nome do país que possui mais restaurantes registrados?

In [310]:
max_country_rest = df2[['restaurant_id', 'country_name']].groupby('country_name')['restaurant_id'].nunique().idxmax()

print(f"O país que possui mais cidades registradas é {max_country_rest}.")


O país que possui mais cidades registradas é India.


3. Qual o nome do país que possui mais restaurantes com o nível de preço igual gourmet
registrados?

In [356]:
gourmet = df2[df2['price_range'] == 'gourmet']
max_country_gourmet = gourmet[['price_range', 'country_name']].groupby('country_name').count().idxmax().iloc[0]

print(f"O país que possui mais restaurantes com o nível de preço gourmet é {max_country_gourmet}.")


O país que possui mais restaurantes com o nível de preço gourmet é United States of America.


4. Qual o nome do país que possui a maior quantidade de tipos de culinária
distintos?

In [357]:
max_country_cuis = df2[['cuisines', 'country_name']].groupby('country_name')['cuisines'].nunique().idxmax()

print(f"O país que possui mais cidades registradas é {max_country_cuis}.")


O país que possui mais cidades registradas é India.


In [309]:
df2.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes', 'country_name'],
      dtype='object')

5. Qual o nome do país que possui a maior quantidade de avaliações feitas?

In [ ]:
max_country_rat = df2[['cuisines', 'country_name']].groupby('country_name')['cuisines'].nunique().idxmax()

print(f"O país que possui mais cidades registradas é {max_country_cuis}.")

6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem
entrega?

7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam
reservas?

8. Qual o nome do país que possui, na média, a maior quantidade de avaliações
registrada?

9. Qual o nome do país que possui, na média, a maior nota média registrada?

10. Qual o nome do país que possui, na média, a menor nota média registrada?

11. Qual a média de preço de um prato para dois por país?

Cidade
1. Qual o nome da cidade que possui mais restaurantes registrados?
2. Qual o nome da cidade que possui mais restaurantes com nota média acima de
4?
3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de
2.5?
4. Qual o nome da cidade que possui o maior valor médio de um prato para dois?
5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária
distintas?
6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem
reservas?
7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem
entregas?
8. Qual o nome da cidade que possui a maior quantidade de restaurantes que
aceitam pedidos online?

Restaurantes
1. Qual o nome do restaurante que possui a maior quantidade de avaliações?
2. Qual o nome do restaurante com a maior nota média?
3. Qual o nome do restaurante que possui o maior valor de uma prato para duas
pessoas?
4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor
média de avaliação?
5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que
possui a maior média de avaliação?
6. Os restaurantes que aceitam pedido online são também, na média, os
restaurantes que mais possuem avaliações registradas?
7. Os restaurantes que fazem reservas são também, na média, os restaurantes que
possuem o maior valor médio de um prato para duas pessoas?
8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América
possuem um valor médio de prato para duas pessoas maior que as churrascarias
americanas (BBQ)?

Tipos de Culinária
1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do
restaurante com a maior média de avaliação?
2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do
restaurante com a menor média de avaliação?
3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do
restaurante com a maior média de avaliação?
4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do
restaurante com a menor média de avaliação?
5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do
restaurante com a maior média de avaliação?
6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do
restaurante com a menor média de avaliação?
7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do
restaurante com a maior média de avaliação?
8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do
restaurante com a menor média de avaliação?
9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do
restaurante com a maior média de avaliação?
10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do
restaurante com a menor média de avaliação?
11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas
pessoas?
12. Qual o tipo de culinária que possui a maior nota média?
13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos
online e fazem entregas?